In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import random
from tqdm import tqdm

In [ ]:
reverseGlove = {0 : '0', 1 : '1', 2 : '2', 3 : '3', 4 : '4', 5 : '5', 6 : '6', 7 : '7', 8 : '8', 9 : '9', 10 : '-', 11 : '<sot>', 12 : '<eot>'}

In [ ]:
def to_hardmax_vec(num, max_num):
  res = np.zeros(max_num)
  res[num] = 1
  return res

glove = {' ': to_hardmax_vec(0,33),
                '/': to_hardmax_vec(1,33),
                '0': to_hardmax_vec(2,33),
                '1': to_hardmax_vec(3,33),
                '2': to_hardmax_vec(4,33),
                '3': to_hardmax_vec(5,33),
                '4': to_hardmax_vec(6,33),
                '5': to_hardmax_vec(7,33),
                '6': to_hardmax_vec(8,33),
                '7': to_hardmax_vec(9,33),
                '8': to_hardmax_vec(10,33),
                '9': to_hardmax_vec(11,33),
                'a': to_hardmax_vec(12,33),
                'b': to_hardmax_vec(13,33),
                'c': to_hardmax_vec(14,33),
                'd': to_hardmax_vec(15,33),
                'e': to_hardmax_vec(16,33),
                'f': to_hardmax_vec(17,33),
                'g': to_hardmax_vec(18,33),
                'h': to_hardmax_vec(19,33),
                'i': to_hardmax_vec(20,33),
                'j': to_hardmax_vec(21,33),
                'l': to_hardmax_vec(22,33),
                'm': to_hardmax_vec(23,33),
                'n': to_hardmax_vec(24,33),
                'o': to_hardmax_vec(25,33),
                'p': to_hardmax_vec(26,33),
                'r': to_hardmax_vec(27,33),
                's': to_hardmax_vec(28,33),
                't': to_hardmax_vec(29,33),
                'u': to_hardmax_vec(30,33),
                'v': to_hardmax_vec(31,33),
                'y': to_hardmax_vec(32,33)
                }

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
days = ['mon','monday','tue','tuesday','wed','wednesday','thu','thursday','fri','friday','sat','saturday','sun','sunday']

In [ ]:
def preProcess(inputWithOutPreProcess):
  wordlist = inputWithOutPreProcess.split()
  for j in wordlist:
    if j in days:
      inputWithOutPreProcess = inputWithOutPreProcess.replace(j, "\b")
  return inputWithOutPreProcess

In [ ]:
path = open('/home/saurav/DLNP Assignment/DLNLP_ASS4/Assignment4aDataset.txt','r')
data = path.readlines()

for i in range(0, len(data)):
  data[i] = data[i].strip()
  data[i] = data[i].lower()
  data[i] = data[i].split(", ")

In [ ]:
df = pd.DataFrame(data = data, columns = ['input', 'output'])
df.head(10)

In [ ]:
inputX = np.char.lower(np.char.replace(list(df['input']),"'",""))
inputY = np.char.lower(np.char.replace(list(df['output']),"'",""))

In [ ]:
def human_date_to_char_list(sample):
  tokens = sample.split()
  filtered_tokens = []
  for i in tokens:
    if i not in days:
      filtered_tokens.append(i)
  char_list = []
  n = len(filtered_tokens)
  for i in range(n):
    curr_char_list = list(filtered_tokens[i])
    if(i != n-1):
      curr_char_list.append(' ')
    char_list.extend(curr_char_list)
  return char_list


def char_list_to_vector(char_list, max_len):
  curr_len = len(char_list)
  for i in range(max_len-curr_len):
    char_list.append(' ')

  vector_date_rep = []

  for i in char_list:
    vector_date_rep.append(glove[i]);
  
  return vector_date_rep


In [ ]:
complete_char_list = [human_date_to_char_list(sample) for sample in inputX]
max_len = 0
for char_list in complete_char_list:
  max_len = max(max_len, len(char_list))

train_X = [char_list_to_vector(char_list,max_len) for char_list in complete_char_list]
train_X = np.array(train_X)



train_Y = []
for sample in inputY:
  temp = list(sample)
  temp.insert(0, '<sot>')
  temp.append('<eot>')
  train_Y.append(temp)
train_Y = np.array(train_Y)



In [ ]:
hardmax_dict = {'0': to_hardmax_vec(0,13),
                '1': to_hardmax_vec(1,13),
                '2': to_hardmax_vec(2,13),
                '3': to_hardmax_vec(3,13),
                '4': to_hardmax_vec(4,13),
                '5': to_hardmax_vec(5,13),
                '6': to_hardmax_vec(6,13),
                '7': to_hardmax_vec(7,13),
                '8': to_hardmax_vec(8,13),
                '9': to_hardmax_vec(9,13),
                '-': to_hardmax_vec(10,13),
                '<sot>': to_hardmax_vec(11,13),
                '<eot>': to_hardmax_vec(12,13)
                }

In [ ]:
y = []
for i in train_Y:
  ss = []
  for j in i:
    ss.append(hardmax_dict[j])
  y.append(ss)
train_Y = np.array(y)


In [ ]:
print(train_X.shape)
print(train_Y.shape)

In [ ]:
train_x = torch.Tensor(train_X)
train_y = torch.Tensor(train_Y)
train_y = train_y.type(torch.LongTensor)
dataset = TensorDataset(train_x,train_y)
training, validation = random_split(dataset, [36000, 4000])
trainLoader = torch.utils.data.DataLoader(dataset=training, batch_size=4000, shuffle=True)
valLoader = torch.utils.data.DataLoader(dataset=validation, batch_size=4000, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, inputSize, outputSize, num_layers, dropout):
        super(Encoder, self).__init__()
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.num_layers = num_layers
        self.dropout = dropout
        self.LSTM = nn.LSTM(inputSize, outputSize, num_layers = num_layers, batch_first = True, bidirectional = False, dropout = dropout)

    def forward(self, x):
        output, (hidden, cell) = self.LSTM(x)
        return output, hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize, num_layers, dropout):
        super(Decoder, self).__init__()
        self.inputSize = inputSize
        self.hiddenSize = hiddenSize
        self.outputSize = outputSize
        self.num_layers = num_layers
        self.dropout = dropout
        self.LSTM = nn.LSTM(inputSize, hiddenSize, num_layers = num_layers, dropout = dropout, batch_first = True, bidirectional = False)
        self.FC = nn.Linear(hiddenSize, inputSize)

    def forward(self, x, hidden, cell):
        output, (hidden, cell) = self.LSTM(x.float(), (hidden, cell))
        output = self.FC(output)
        return output, hidden, cell

In [ ]:
class AttentionDecoder(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize, num_layers, dropout):
        super(AttentionDecoder, self).__init__()
        self.inputSize = inputSize
        self.hiddenSize = hiddenSize
        self.outputSize = outputSize
        self.num_layers = num_layers
        self.dropout = dropout

        self.linear1 = nn.Linear(inputSize, hiddenSize)
        self.attn = nn.Linear(self.hiddenSize * 2, 17)
        self.attn_combine = nn.Linear(self.hiddenSize*2, self.inputSize)

        self.lstm = nn.LSTM(inputSize, hiddenSize, num_layers, batch_first = True, dropout = dropout, bidirectional = False)   
        
        self.linear = nn.Linear(hiddenSize, outputSize)
            
    def forward(self, input, hidden, cell, encoder_outputs):  
        
        # print(hidden.shape)
        out = self.linear1(input)
        # hidden = hidden.squeeze(0)
        # hidden = hidden.unsqueeze(1)
        # print(out.shape, hidden.shape, torch.cat((out, hidden), 2).shape)
        attn_weights = F.softmax(self.attn(torch.cat((out, (hidden.squeeze(0)).unsqueeze(1)), 2)), dim=1)
        # print(attn_weights.shape)
        attn_applied = torch.bmm(attn_weights, encoder_outputs)
        # print(attn_applied.shape, out.shape, torch.cat((out, attn_applied), 2).shape)
        output = torch.cat((out, attn_applied), 2)
        output = self.attn_combine(output)
        output = F.relu(output)
        # print(output.shape)
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
        output = self.linear(output)
        return output, hidden, cell



In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, input, target, teacherForceRatio = 0.5):
        batchSize = input.shape[0]
        targetLength = target.shape[1]
        targetVocabSize = target.shape[2]
        outputTensor = torch.zeros(batchSize, targetLength, targetVocabSize).to(device)

        encoderOutput, hidden, cell = self.encoder(input)
        # print(hidden.shape)
        x = target[:, 0, :].unsqueeze(1)
        for t in range(1, targetLength):
            output, hidden, cell = self.decoder(x.float(), hidden, cell, encoderOutput)
            outputTensor[:, t-1, :] = output.squeeze(dim = 1)
            x = target[:, t, :].unsqueeze(1) if random.random() < teacherForceRatio else output
        
        return outputTensor

In [ ]:
epochs, lr = 30, 0.01
encoderNetwork = Encoder(inputSize = 33, outputSize = 256, num_layers = 1, dropout = 0).to(device)
decoderNetwork = AttentionDecoder(inputSize = 13, hiddenSize = 256, outputSize = 13, num_layers = 1, dropout = 0).to(device)
model = Seq2Seq(encoder = encoderNetwork, decoder = decoderNetwork).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr = lr)

In [ ]:
trainLoss = []
valLoss = []
for epoch in range(epochs):
    tLoss = 0
    vLoss = 0
    count = 0
    model.train()
    for x, y in tqdm(trainLoader):
        x, y = x.to(device), y.to(device) 
        output = model(x, y)
        optimiser.zero_grad()
        loss = criterion(output[:, 1:-1, :], y[:, 1:-1, :].float())
        loss.backward()
        optimiser.step()
        tLoss += loss.item()
    
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(valLoader):
            x, y = x.to(device), y.to(device) 
            output = model(x, y)
            loss = criterion(output[:, 1:-1, :], y[:, 1:-1, :].float())
            vLoss += loss.item()

            predictedY = torch.argmax(output, dim = 2)
            trueY = torch.argmax(y, dim = 2)
            for batch in range(40):
                predictedIndexList = list(predictedY[batch].cpu().detach().numpy())
                predictedDate = ''.join(reverseGlove[i] for i in predictedIndexList[1: -1])
                trueIndexList = list(trueY[batch].cpu().detach().numpy())
                trueDate = ''.join(reverseGlove[i] for i in trueIndexList[1: -1])
                if(predictedDate == trueDate):
                    count += 1
    
    acc = count
    print(f'Epoch {epoch+1} Training Loss: {tLoss/len(trainLoader)} Validation Loss: {vLoss/len(valLoader)} accuracy: {acc}')

In [ ]:
for x, y in trainLoader:
    x, y = x.to(device), y.to(device) 
    output = model(x, y)
    predictedY = torch.argmax(output, dim = 2)
    trueY = torch.argmax(y, dim = 2)
    for batch in range(10):
        predictedIndexList = list(predictedY[batch].cpu().detach().numpy())
        predictedDate = ''.join(reverseGlove[i] for i in predictedIndexList[1: -1])
        trueIndexList = list(trueY[batch].cpu().detach().numpy())
        trueDate = ''.join(reverseGlove[i] for i in trueIndexList[1: -1])
        print(trueDate, predictedDate)
    break